In [ ]:
import pandas as pd
import os
path = 'output'
device_file = '../traffic-analysis/devices-ipv6.txt'
device_category_file = '../traffic-analysis/ipv6-device-category.csv'
merge_files = [('appletv.csv', 'appletv-wifi.csv'), ('samsungtv65-wifi.csv', 'samsungtv65-wired.csv')]

for merged in merge_files:
    df1 = pd.read_csv(os.path.join(path, merged[0]))
    df2 = pd.read_csv(os.path.join(path, merged[1]))
    # merge two dfs, duplicates rows are removed
    df = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
    df.to_csv(os.path.join(path, 'new' + merged[0]), index=False)

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import json

ipv6_only_functionaly_devices = ['facebook-portal-mini', 'google-home-mini', 'google-nest-mini1', 'nest-hub', 'nest-hub-max', 'appletv', 'chromecast-googletv', 'tivostream']

def transpose_df(df, index_column_name=['Category', 'Count']):
    df.set_index(index_column_name, inplace=True)
    df = df.T
    # df.reset_index(inplace=True)
    df = df.reset_index().rename(columns={'index': index_column_name[0], '': index_column_name[1]})
    return df

def add_total_row(df, index=2):
    df_toal = df.copy()
    for column in df.columns[index:]:
        # print(column)
        df_toal[column] = df_toal[column].astype(int)
    df_toal.loc['Total'] = df_toal.sum(numeric_only=True, axis=0) # .round().astype(int)
    # print(df_toal)
    for i in range(index):
        df_toal.iloc[-1, i] = 'Total'
    
    return df_toal

def add_percentage(df):
    df_percentage = df.copy()
    # df_percentage.reset_index(inplace=False)
    total_num = 93
    df_percentage['Percentage'] = round((100* df_percentage['Total'].astype(int) / total_num),1).astype(str) + '\%'
    return df_percentage

path = 'output'
device_file = '../traffic-analysis/devices-ipv6.txt'
device_category_file = '../traffic-analysis/ipv6-device-category.csv'
output_path = 'analysis_results_nonfunctional'
# output_path = 'analysis_results_functional'
if not os.path.exists(output_path):
    os.makedirs(output_path)

with open(device_file, 'r') as f:
    devices = f.readlines()
    if output_path == 'analysis_results_functional':
        device_list = [x.strip() for x in devices if x.strip() != '' and x.strip() in ipv6_only_functionaly_devices]
    else:
        device_list = [x.strip() for x in devices if x.strip() != '' and x.strip() not in ipv6_only_functionaly_devices]
print(len(device_list))

# read all csv files in the output folder
files = [f for f in os.listdir(path) if f.endswith('.csv') and f.split('.')[0] in device_list]
print(len(files))

columns = ['Device', 'Category', 'Manufacturer', 'Domain number', 'AAAA Response', 'IPv6-v4 diff']
df = pd.DataFrame(columns=columns)
df['Device'] = device_list
device_category = {}
device_manufacturer = {}
df_category = pd.read_csv(device_category_file)
for index, row in df_category.iterrows():
    device_category[row['NAME']] = row['CATEGORY']
    device_manufacturer[row['NAME']] = row['MANUFACTURER']
df['Category'] = df['Device'].map(device_category)
df['Manufacturer'] = df['Device'].map(device_manufacturer)

for file in files:
    df_temp = pd.read_csv(os.path.join(path, file))
    ipv6_response_df = df_temp[df_temp['Resolver'] == '2001:4860:4860::8888']
    ipv4_response_df = df_temp[df_temp['Resolver'] == '8.8.8.8']
    domain_number = df_temp['Domain'].nunique()
    domain_number2 = ipv6_response_df['Domain'].nunique()
    if domain_number!=domain_number2:
        print('Domain number mismatch', file)
    aaaa_response = ipv6_response_df[ipv6_response_df['Response Type'] == 'no error AAAA response']['Domain'].nunique()
    # Difference in response from IPv6 and IPv4 DNS resolver
    aaaa_response_v4 = ipv4_response_df[ipv4_response_df['Response Type'] == 'no error AAAA response']['Domain'].nunique()
    ipv6_v4_diff = aaaa_response - aaaa_response_v4
    if aaaa_response != aaaa_response_v4:
        if aaaa_response < aaaa_response_v4:
            mismatch_domain = set(ipv4_response_df[ipv4_response_df['Response Type'] == 'no error AAAA response']['Domain'].values) - set(ipv6_response_df[ipv6_response_df['Response Type'] == 'no error AAAA response']['Domain'].values)
        else:
            mismatch_domain = set(ipv6_response_df[ipv6_response_df['Response Type'] == 'no error AAAA response']['Domain'].values) - set(ipv4_response_df[ipv4_response_df['Response Type'] == 'no error AAAA response']['Domain'].values)
        
        print('AAAA response mismatch', file, mismatch_domain)
    
    df.loc[df['Device'] == file.split('.')[0], 'Domain number'] = domain_number
    df.loc[df['Device'] == file.split('.')[0], 'AAAA Response'] = aaaa_response
    df.loc[df['Device'] == file.split('.')[0], 'IPv6-v4 diff'] = ipv6_v4_diff

df.to_csv(os.path.join(output_path,'_summary.csv'), index=False)
df_per_category = df.copy().iloc[:,1:] # remove device column
df_per_category = df_per_category.drop('Manufacturer', axis=1)
# group rows and remove duplicates
for column in columns[3:]:
    df_per_category[column] = df_per_category.groupby('Category')[column].transform('sum')
df_per_category['Count'] = df_per_category.groupby('Category')['Category'].transform('size')
# Move 'Count' column to the second position
cols = df_per_category.columns.tolist()
cols.insert(1, cols.pop(cols.index('Count')))
df_per_category = df_per_category[cols]
df_per_category = df_per_category.drop_duplicates(subset=['Category', 'Count'])
df_per_category.to_csv(os.path.join(output_path,'_summary_per_category.csv'), index=False)

df_per_category = add_total_row(df_per_category, index=1)
df_percentage = df_per_category.copy()
for column in df_percentage.columns[1:]:
    df_percentage[column] = df_percentage[column].astype(int)

df_percentage['Percentage'] = round((100.0* df_percentage['AAAA Response'] / df_percentage['Domain number']),1).astype(str) + '\%'
df_percentage.to_csv(os.path.join(output_path,'_summary_per_category_total.csv'), index=False)
df_percentage_T = transpose_df(df_percentage)
df_percentage_T.to_csv(os.path.join(output_path,'_summary_per_category_total_T.csv'), index=False)

df_per_manufacturer = df.copy().iloc[:,1:] # remove device column
df_per_manufacturer = df_per_manufacturer.drop('Category', axis=1)
# group rows and remove duplicates
for column in columns[3:]:
    df_per_manufacturer[column] = df_per_manufacturer.groupby('Manufacturer')[column].transform('sum')
df_per_manufacturer['Count'] = df_per_manufacturer.groupby('Manufacturer')['Manufacturer'].transform('size')
# Move 'Count' column to the second position
cols = df_per_manufacturer.columns.tolist()
cols.insert(1, cols.pop(cols.index('Count')))
df_per_manufacturer = df_per_manufacturer[cols]
df_per_manufacturer = df_per_manufacturer.drop_duplicates(subset=['Manufacturer', 'Count'])
df_per_manufacturer.to_csv( os.path.join(output_path,'_summary_per_manufacturer.csv'), index=False)

df_per_manufacturer = add_total_row(df_per_manufacturer, index=1)
df_percentage = df_per_manufacturer.copy()
for column in df_percentage.columns[1:]:
    df_percentage[column] = df_percentage[column].astype(int)
df_percentage['Percentage'] = round((100.0* df_percentage['AAAA Response'] / df_percentage['Domain number']),1).astype(str) + '\%'
df_percentage.to_csv(os.path.join(output_path,'_summary_per_manufacturer_total.csv'), index=False)
df_percentage_T = transpose_df(df_percentage, index_column_name=['Manufacturer', 'Count'])
df_percentage_T.to_csv(os.path.join(output_path,'_summary_per_manufacturer_total_T.csv'), index=False)

In [5]:
import pandas as pd
import os

def merge_csv_files(path, csv_files):
    # Read and store each DataFrame in a list
    dfs = [pd.read_csv(f'{csv_file}') for csv_file in csv_files]

    # Concatenate the DataFrames
    df = pd.concat(dfs)

    # Reset the index
    df = df.reset_index(drop=True)

    return df

path = 'merged'
if not os.path.exists(path):
    os.system('mkdir -pv %s' % path)
csv_files = [
    'analysis_results_functional/_summary_per_category_total.csv',
    'analysis_results_nonfunctional/_summary_per_category_total.csv',
]

df = merge_csv_files(path, csv_files)
df.to_csv(os.path.join(path, 'dns_aaaa_merged.csv'), index=False)

csv_files = [
    'analysis_results_functional/_summary_per_manufacturer_total.csv',
    'analysis_results_nonfunctional/_summary_per_manufacturer_total.csv',
]
df_functional = pd.read_csv(csv_files[0])
df_non_functional = pd.read_csv(csv_files[1])
df_non_functional = df_non_functional[df_non_functional['Count'] >= 3]
df = pd.concat([df_functional, df_non_functional]).reset_index(drop=True)
df.to_csv(os.path.join(path, 'dns_aaaa_manufacturer_merged.csv'), index=False)